In [ ]:
import numpy as np
import cv2
from cv2 import dnn

In [2]:
proto_file = r"C:/Users/Srinaath/Desktop/Image Coloriser/colorization_deploy_v2.prototxt"
model_file = r"C:/Users/Srinaath/Desktop/Image Coloriser/colorization_release_v2.caffemodel"
hull_pts = r"C:/Users/Srinaath/Desktop/Image Coloriser/pts_in_hull.npy"
img_path = r"C:/Users/Srinaath/Downloads/wilsoncolor.jpeg"

In [3]:
net = dnn.readNetFromCaffe(proto_file,model_file)
kernel = np.load(hull_pts)

In [4]:
img = cv2.imread(img_path)
scaled = img.astype("float32") / 255.0
lab_img = cv2.cvtColor(scaled, cv2.COLOR_BGR2LAB)

In [5]:
class8 = net.getLayerId("class8_ab")
conv8 = net.getLayerId("conv8_313_rh")
pts = kernel.transpose().reshape(2, 313, 1, 1)
net.getLayer(class8).blobs = [pts.astype("float32")]
net.getLayer(conv8).blobs = [np.full([1, 313], 2.606, dtype="float32")]

In [6]:
resized = cv2.resize(lab_img, (224, 224))

In [7]:
L = cv2.split(resized)[0]

In [8]:
L -= 50

In [9]:
net.setInput(cv2.dnn.blobFromImage(L))
ab_channel = net.forward()[0, :, :, :].transpose((1, 2, 0))

In [10]:
ab_channel = cv2.resize(ab_channel, (img.shape[1], img.shape[0]))

In [11]:
L = cv2.split(lab_img)[0]

In [12]:
colorized = np.concatenate((L[:, :, np.newaxis], ab_channel), axis=2)

In [13]:
colorized = cv2.cvtColor(colorized, cv2.COLOR_LAB2BGR)
colorized = np.clip(colorized, 0, 1)

In [14]:
colorized = (255 * colorized).astype("uint8")

In [15]:
img = cv2.resize(img,(640,640))
colorized = cv2.resize(colorized,(640,640))

In [16]:
result = cv2.hconcat([img,colorized])

In [17]:
cv2.imshow("Grayscale -> Colour", result)

In [ ]:
cv2.waitKey(0)